## cleaning notebook

- do cleaning
- basically spellcheck got rid of ~1000 unique words which is minor improvement but removing slang, stopwords, punctuation, urls should help even more
- jamspell was the best spellchecker between textblob, symspell, and jamspell but its also not perfect sadly

Notes about below code:
- reformatted the slang in txt called slang_edit.txt
- replaced all slang with definition
- removed punctuation using string.punctuation, then removed additional randowm punctuation like "''", "--", "``", "..."
- replaced website URLs with "website"
- removed stopwords
- in total removed ~600 unique words

Decisions to be made/things to fix:
- are we happy with how contractions are being treated?
- do we want to remove "I" and/or "you" from the stopwords list?


Iris 
- added contractions to slang list -- ex. 'can't' : 'can not' 

Lydia
- removed {'i', 'you', 'they', 'them', 'me', 'what', 'want', 'go', 'of', 'to', 'let', 'get', 'got', 'not'} from stop word list? My vote is to keep words like "i" and "you" + words that do add context/sentiment like "not" 

Anna
- added "propane" (actually propname after autocorrect) to stopwords? how do we feel about that... 

Diane
- rearranged order of cleaning slightly 

In [18]:
# read in any necessary libs
import pandas as pd 
# nltk langs
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

In [14]:
# use the spellchecked csv file: 'en_personality_spellcheck.csv'
df = pd.read_csv('en_personality_spellcheck.csv')
#df.head()

In [20]:
slang_dict = {}
#read in slang txt
with open("slang_edit.txt", "r") as file:
    for line in file:
        line = line.strip()
        if line:
            slang_word, meaning = line.split(':')
            slang_dict[slang_word.strip().strip("'")] = meaning.strip().strip("'")

        
def replace_slang(tokens):
    return [slang_dict[token] if token in slang_dict.keys() else token for token in tokens]

Remove stopwords and punctuation

In [21]:
#remove punctuation. replace all ... with >3 dots with ... and replace all URLs with "website"
punc = string.punctuation #removed < so that hearts wouldn't be removed
punc = punc.replace("<","")

def remove_punctuation(tokens):
    # Remove any "//" from tokens and split the token into two tokens
    tokens = ['website' if (token.count('www.') == 1 or token.count('.com')==1 or token.count('.org')==1) else token for token in tokens]
    tokens = [subtoken for token in tokens for subtoken in token.split("//")]
    tokens = [token.replace("-", "") for token in tokens]
    tokens = [token.replace(" ", "") for token in tokens]
    tokens = [token.replace(".", "") for token in tokens] #might want to leave in periods at ends of sentences...
    tokens = ['...' if token.count('.') > 3 else token for token in tokens]
    tokens = [token for token in tokens if token not in ["''", "--", "``", "n't","..."]]
    tokens = [re.sub(r'(.)\1{3,}', r'\1\1', token) for token in tokens]
    # tokens = ['website' if (token.count('www.') == 1 or token.count('.com')==1 or token.count('.org')==1) else token for token in tokens]
    return [token for token in tokens if token not in punc]
#remove preset stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    add_stop_words = {"n't", "'s", "'m"}
    stop_words.update(add_stop_words)
    words_to_remove = {'i', 'you', 'they', 'them', 'me', 'what', 'want', 'go', 'of', 'to', 'let', 'get', 'got', 'not'}
    for word in words_to_remove:
        if word in stop_words:
            stop_words.remove(word)
    return [token for token in tokens if token.lower() not in stop_words]

def preprocess_text(text):
    tokens = word_tokenize(text) #' '.join(text)
    tokens = replace_slang(tokens) #replace slang with defs
    tokens = remove_punctuation(tokens) #remove punctuation
    combined_tokens = []
    i = 0
    while i < len(tokens): #recombine hearts
        if tokens[i] == '<' and i+1 < len(tokens) and tokens[i+1] == '3':
            combined_tokens.append('love')
            i += 2
        else:
            combined_tokens.append(tokens[i])
            i += 1
    return ' '.join(combined_tokens)

df['corrected_StopsPuncSlang'] = df['corrected'].apply(preprocess_text)

In [5]:
df['corrected_StopsPuncSlang']

0                              likes the sound of thunder
1       is so sleepy it 's not even funny that 's she ...
2       is more and wants the knot of muscles at the b...
3               likes how the day sounds in this new song
4                                            is home love
                              ...                        
9115                                    ahh it 's snowing
9116                                           its friday
9117                                             go likes
9118                                      best night ever
9119                                                  red
Name: corrected_StopsPuncSlang, Length: 9120, dtype: object

In [6]:
before = df['STATUS'].apply(word_tokenize).explode().unique()
print("Original # unique words", len(before))
after = df['corrected'].apply(word_tokenize).explode().unique()
print("Spellchecked # unique words:", len(after))
new = df['corrected_StopsPuncSlang'].apply(word_tokenize).explode().unique()
print("Stopwords, slang, and punct removed unique words:", len(new))
print("Original total words:", len(df['STATUS'].apply(word_tokenize).explode()))
print("Post-processing total words:", len(df['corrected_StopsPuncSlang'].apply(word_tokenize).explode()))

Original # unique words 15315
Spellchecked # unique words: 14461
Stopwords, slang, and punct removed unique words: 13903
Original total words: 174366
Post-processing total words: 142121


In [14]:
df['corrected_StopsPuncSlang'].head(30)

0                            likes the sound of thunder
1     is so sleepy it 's not even funny that 's she ...
2     is more and wants the knot of muscles at the b...
3             likes how the day sounds in this new song
4                                          is home love
5                                               website
6     saw a nun zombie and liked it also name tentac...
7     is in kentucky 421 miles into her 1100 mile jo...
8     was about to finish a digital painting before ...
9     is celebrating her new haircut by listening to...
10                     has a crush on the green lantern
11                               has magic on the brain
12    saw transformers up and year one this week goo...
13    who wants to meet up on schedule pickup day at...
14         desires the thrill of inspiration also money
15                          is going to be at 9:30 yeah
16    is reading admiring her permit and occasionall...
17    thinks intangibility should be an option i

In [15]:
df.to_csv('cleanedCOPY.csv', index=False)

The following code checks the number of unique users, and that the personalities given to each user is the same among all of their posts.

In [ ]:
# Print the number of unique users
print(f"There are {len(df['#AUTHID'].unique())} unique users.")

columns_to_check = ["sEXT", "sNEU", "sAGR", "sCON", "sOPN", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]

# Grouping by '#AUTHID' and checking if all rows within each group have the same values in the specified columns
result = df.groupby('#AUTHID').apply(lambda x: (x[columns_to_check] == x[columns_to_check].iloc[0]).all().all())

# Checking if any of the results are False
any_false = not result.all()

# Displaying the result
print(f"Are there any columns that have different typings: {any_false}.")

The following code will merge all of the text for each user on the column that has cleaned punctuation. The other columns will also be transferred as there is no different data among users.

In [ ]:
# Create a new DataFrame to store the result
grouped = pd.DataFrame(columns=['#AUTHID', 'text', 'sNEU', 'cNEU'])

# Iterate over unique #AUTHIDs
for authid in df['#AUTHID'].unique():
    # Filter the DataFrame for the current #AUTHID
    temp_df = df[df['#AUTHID'] == authid]

    # Fill NaN values in 'corrected_StopsPuncSlang' with empty strings
    temp_df['corrected_StopsPuncSlang'] = temp_df['corrected_StopsPuncSlang'].fillna('')

    # Concatenate the 'corrected_StopsPuncSlang' text
    text = ' '.join(temp_df['corrected_StopsPuncSlang'])
    # Choose Neuropathy to copy
    sNEU_value = temp_df['sNEU'].iloc[0]  # Assuming it's the same for all rows of the same #AUTHID

    cNEU_value = temp_df['cNEU'].iloc[0]

    # Create a DataFrame with the current #AUTHID, concatenated text, and copied 'sEXT' value
    result_df = pd.DataFrame({'#AUTHID': [authid], 'text': [text], 'sNEU': [sNEU_value], 'cNEU': [cNEU_value]})
    # Concatenate the result DataFrame with the new_df
    grouped = pd.concat([grouped, result_df], ignore_index=True)

# Displaying the new DataFrame
grouped.head()

In [ ]:
#split up grouped text into rows of max 100 words each
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//50 >0:
    n = len(text1.split())//50
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:100]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*50:w*50 + 100]
      l_total.append(" ".join(l_parcial))
  return l_total

grouped['textsplt'] = grouped['text'].apply(get_split)

In [ ]:
# ungroup grouped text lol
group = grouped.explode('textsplt')

In [ ]:
group.to_csv("group.csv")